In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [3]:
# import units needed in the model
s = UNITS.second
N = UNITS.newton
kg = UNITS.kilogram
m = UNITS.meter


<Unit('meter')>

In [4]:
# set up initial values

v = Vector(0, 0) * m / s       # velocity vector of the frog
p_frog = Vector(0, 0) * m      # position vector of the frog
p_lily = Vector(0, 0) * m      # position vector of the lilypad

# By setting uo initial velocity manually, a certain maximum height is determined
# We can sweep through a series of initial velocity

init = State(x_frog = p_frog.x,
             y_frog = p_frog.y,
             x_lily = p_lily.x,
             y_lily = p_lily.y,
             vx = v.x,
             vy = v.y)

,values
x_frog,0.0 meter
y_frog,0.0 meter
x_lily,0.0 meter
y_lily,0.0 meter
vx,0.0 meter / second
vy,0.0 meter / second


In [7]:
# set up system values

# the frog is treated as point mass
system = System(init=init,
                g_a = 9.8 * m / s **2,    # gravitational acceleration
                m_frog = 0.1 * kg,        # mass of the frog
                r_lilypad = 0.1 * m,      # radius of lilypad
                ρ = 1e3 * kg / m**3,      # density of water 
                t_0 = 0 * s,
                t_end = 300 * s)

,values
init,x_frog 0.0 meter y_frog ...
g_a,9.8 meter / second ** 2
m_frog,0.1 kilogram
r_lilypad,0.1 meter
ρ,1000.0 kilogram / meter ** 3
t_0,0 second
t_end,300 second


I think we can model the lilypad as a sphere and apply the equation of spherical object submerged in water. Thus, if the volumn in water is greater than a certain value that water starts to leak in the lilypad, we know that the lilypad will sink. 

In [ ]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, x velocity, y velocity)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (vx, vy, ax, ay)
    """
    x, y, vx, vy = state
    unpack(system)

    V = Vector(vx, vy)    
    
    

    
    return vx, vy, a.x, a.y